In [16]:
import tensorflow as tf
import numpy as np
import tensorflow_datasets as tfds


In [17]:
tf.__version__



'2.3.0'

In [18]:
(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

In [19]:
ds_train

<_OptionsDataset shapes: ((28, 28, 1), ()), types: (tf.uint8, tf.int64)>

In [4]:
def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(128)
ds_train = ds_train.prefetch(tf.data.experimental.AUTOTUNE)

In [5]:
ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_test = ds_test.batch(128)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.experimental.AUTOTUNE)

In [14]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(
    filters=10,
    kernel_size=3,
    input_shape=( 28, 28, 1),
    activation="relu"
    ),
  tf.keras.layers.Conv2D(10,3, activation='relu'),
  tf.keras.layers.MaxPool2D(),
  tf.keras.layers.Conv2D(10,3, activation='relu'),
  tf.keras.layers.Conv2D(10,3, activation='relu'),
  tf.keras.layers.MaxPool2D(),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(10)
])
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)

model.fit(
    ds_train,
    epochs=5,
    validation_data=ds_test,
)

Epoch 1/5
469/469 [==============================] - 13s 28ms/step - loss: 0.4730 - sparse_categorical_accuracy: 0.8532 - val_loss: 0.1719 - val_sparse_categorical_accuracy: 0.9479
Epoch 2/5
469/469 [==============================] - 13s 28ms/step - loss: 0.1377 - sparse_categorical_accuracy: 0.9584 - val_loss: 0.1004 - val_sparse_categorical_accuracy: 0.9686
Epoch 3/5
469/469 [==============================] - 13s 28ms/step - loss: 0.0964 - sparse_categorical_accuracy: 0.9713 - val_loss: 0.0677 - val_sparse_categorical_accuracy: 0.9781
Epoch 4/5
469/469 [==============================] - 13s 28ms/step - loss: 0.0809 - sparse_categorical_accuracy: 0.9751 - val_loss: 0.0735 - val_sparse_categorical_accuracy: 0.9755
Epoch 5/5
469/469 [==============================] - 13s 28ms/step - loss: 0.0687 - sparse_categorical_accuracy: 0.9783 - val_loss: 0.0509 - val_sparse_categorical_accuracy: 0.9829


In [7]:
model.evaluate(ds_test )

79/79 [==============================] - 0s 645us/step - loss: 0.0784 - sparse_categorical_accuracy: 0.9802


[0.07840216159820557, 0.9801999926567078]

In [8]:
ds_test

<PrefetchDataset shapes: ((None, 28, 28, 1), (None,)), types: (tf.float32, tf.int64)>